In [ ]:
import os
from datetime import datetime
from .utils import get_proxy_settings
from .proxy import fetch_feed_content_with_proxy
from .feed_parser import retry_with_backoff, parse_feed

def process_feeds(feed_urls):
    """
    Process multiple RSS feeds and save extracted metadata for each feed.

    Args:
        feed_urls (list): List of RSS feed URLs.

    Returns:
        None
    """
    # Get proxy settings
    proxy_settings = get_proxy_settings()
    app_proxy = proxy_settings["app_proxy"]
    username = proxy_settings["username"]
    password = proxy_settings["password"]

    # Prepare directory for saving metadata
    timestamp = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")
    output_dir = "feed_metadata"
    os.makedirs(output_dir, exist_ok=True)

    for feed_url in feed_urls:
        print(f"Processing feed: {feed_url}")
        try:
            # Fetch the feed content
            feed_content = retry_with_backoff(feed_url, app_proxy, username, password)
            if feed_content:
                # Parse the feed
                parsed_feed = parse_feed(feed_content)
                if parsed_feed:
                    # Prepare metadata to save
                    domain = feed_url.split("/")[2]
                    filename = f"metadata_{domain}_{timestamp}.json"
                    filepath = os.path.join(output_dir, filename)

                    # Save metadata
                    with open(filepath, "w", encoding="utf-8") as f:
                        f.write(parsed_feed)  # Ensure parse_feed returns serialized JSON
                    print(f"Metadata saved to: {filepath}")
                else:
                    print(f"Failed to parse feed: {feed_url}")
            else:
                print(f"Failed to fetch feed: {feed_url}")
        except Exception as e:
            print(f"Error processing feed {feed_url}: {e}")
